---
### Building a database for crime reports
---

__Data:__ boston.csv

__Aim:__ To create a database (crimes_db) with a table (boston_crimes), with appropriate datatypes for storing the data from the boston.csv file. 

- The table will be created inside a schema named crimes, including creating the readonly and readwrite groups with the appropriate privileges. 

In [1]:
import psycopg2 as sql, csv

basefile = 'boston.csv'

__Create the Database__

In [2]:
conn = sql.connect(dbname = 'dq', user = 'dq')
conn.autocommit = True
cu = conn.cursor()

# To allow the kernel to re-run the build
cu.execute('''DROP DATABASE crimes_db;''')

cu.execute('''DROP USER readonly;''')
cu.execute('''DROP USER readwrite;''')

cu.execute('''DROP USER data_analyst;''')
cu.execute('''DROP USER data_scientist;''')

# Create the (new) database
cu.execute('''CREATE DATABASE crimes_db;''')

conn.close()

In [3]:
conn = sql.connect(dbname = 'crimes_db', user = 'dq')
cu = conn.cursor()
cu.execute('''
    CREATE SCHEMA crimes;
''')

__Import CSV and gather column names / sample data__

In [4]:
table = list()
with open(basefile, 'r') as f:
    read = csv.reader(f)
    for row in read:
        table.append(row)
col_headers = table[0]
first_row = table[1]

In [5]:
print(col_headers, '\n', first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long'] 
 ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


__Create Auxiliary Function__

In [6]:
def get_col_set(file, col_index):
    col_data = set()
    
    with open(file, 'r') as f:
        next(f) # skip header row
        reader = csv.reader(f)
        for row in reader:
            col_data.add(row[col_index])

    return col_data

In [7]:
for i in range(0,7):
    print(col_headers[i], len(get_col_set(basefile, i)))

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177
long 18177


__Finding Max Length__

In [8]:
col_headers

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

In [9]:
max_length = 0
max_entry = 0
for i in get_col_set(basefile, 2):
    if len(i) > max_length:
        max_length = len(i)
        max_entry = i
print(max_entry,':', max_length)

RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUTSIDE BOSTON) : 58


__Create the Table__

In [10]:
print(
    col_headers,
    '\n',
    first_row
)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long'] 
 ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


Columns with a low number of distinct values tend to be good candidates for enumerated types. Another important aspect is to know the longest word in any column containing textual data.

We will use the following types:

    Incident No - serial - PRIMARY KEY
    Offence Code - varchar(4)
    Description - varchar(100) - increase to allow for future changes
    Date - date
    Day of week - enumerate
    Lat - decimal(10, 8) 
    Long - decimal(11,8) 

In [11]:
offense_code_values = get_col_set(basefile, 1)
max_length = 0
max_value = 0 
for row in offense_code_values:
    if len(row) > max_length:
        max_length = len(row)
    if int(row) > max_value:
        max_value = int(row)
print(max_length, max_value)

4 3831


In [12]:
def get_scale(col_value_set):    
    max_scale = 0
    for row in col_value_set:        
        scale = 0
        for idx, r in enumerate(row):            
            if r == ".":
                scale = len(row)- idx - 1        
        if scale > max_scale:
            max_scale = scale
    return(max_scale)

lat_values = get_col_set(basefile, 5)
long_values =  get_col_set(basefile, 6)
lat_max_scale = get_scale(lat_values)
long_max_scale = get_scale(long_values)
print(lat_max_scale, long_max_scale)

8 8


Create boston_crimes table inside the crimes schema with the appropriate data type for each column.

In [13]:
cu.execute("""CREATE TYPE day_of_the_week_enum AS ENUM(
'Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday');""")

cu.execute('''
           CREATE TABLE crimes.boston_crimes(
           incident_number serial PRIMARY KEY,
           offence_code varchar(4),
           description varchar(100),
           date date,
           day_of_the_week day_of_the_week_enum,
           lat decimal(10,8),
           long decimal(11,8)
           );
           ''')
conn.commit()

In [14]:
#checking the result
cu.execute("SELECT * FROM crimes.boston_crimes LIMIT 0;")
cu.description

(Column(name='incident_number', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='offence_code', type_code=1043, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='description', type_code=1043, display_size=None, internal_size=100, precision=None, scale=None, null_ok=None),
 Column(name='date', type_code=1082, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='day_of_the_week', type_code=16623, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='lat', type_code=1700, display_size=None, internal_size=10, precision=10, scale=8, null_ok=None),
 Column(name='long', type_code=1700, display_size=None, internal_size=11, precision=11, scale=8, null_ok=None))

__Loading the Data__

In [15]:
with open(basefile, 'r') as f:
    cu.copy_expert("COPY crimes.boston_crimes FROM STDIN CSV HEADER;", f)

conn.commit()

In [16]:
# Check
cu.execute('SELECT * FROM crimes.boston_crimes LIMIT 5;')
cu.fetchall()

[(1,
  '619',
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  Decimal('42.35779134'),
  Decimal('-71.13937053')),
 (2,
  '1402',
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  Decimal('42.30682138'),
  Decimal('-71.06030035')),
 (3,
  '3410',
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.34658879'),
  Decimal('-71.07242943')),
 (4,
  '3114',
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.33418175'),
  Decimal('-71.07866441')),
 (5,
  '3114',
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.27536542'),
  Decimal('-71.09036101'))]

__Manage Access Privileges__

In [17]:
cu.execute('REVOKE ALL ON SCHEMA public FROM public;')
cu.execute('REVOKE ALL ON DATABASE crimes_db FROM public;')
conn.commit()

__Create User Groups__

1. ReadOnly - Data Analysts etc, access to view data only.
2. ReadWrite - Data Scientists, acess to view and modify the data, including add more columns / rows.
3. Admin - Data Engineer, full control over infrastructure of database (current account)

In [18]:
cu.execute('''CREATE GROUP readonly NOLOGIN;''')
cu.execute('''CREATE GROUP readwrite NOLOGIN;''')

cu.execute('''GRANT CONNECT ON DATABASE crimes_db to readonly;''')
cu.execute('''GRANT CONNECT ON DATABASE crimes_db to readwrite;''')

cu.execute('''GRANT USAGE ON SCHEMA crimes to readonly;''')
cu.execute('''GRANT USAGE ON SCHEMA crimes to readwrite;''')

cu.execute('''GRANT SELECT ON ALL TABLES IN SCHEMA crimes to readonly;''')
cu.execute('''GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes to readwrite;''')

conn.commit()

In [19]:
cu.execute('''CREATE USER data_analyst WITH PASSWORD 'secret1';''')
cu.execute('''GRANT data_analyst TO readonly;''')

cu.execute('''CREATE USER data_scientist WITH PASSWORD 'secret2';''')
cu.execute('''GRANT data_scientist TO readwrite;''')

conn.commit()

__Check Privileges__

In [20]:
cu.execute("""SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readonly';""")
cu.fetchall()

[('readonly', 'SELECT')]

In [21]:
cu.execute("""SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';""")
cu.fetchall()

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

In [22]:
cu.execute("""SELECT rolsuper, rolcanlogin, rolcreaterole, rolcreatedb
    FROM pg_roles
    WHERE rolname = 'readonly';""")
cu.fetchall()

[(False, False, False, False)]

In [23]:
cu.execute("""SELECT rolsuper, rolcanlogin, rolcreaterole, rolcreatedb
    FROM pg_roles
    WHERE rolname = 'readwrite';""")
cu.fetchall()

[(False, False, False, False)]

In [24]:
cu.execute("""SELECT rolsuper, rolcanlogin, rolcreaterole, rolcreatedb, rolpassword
    FROM pg_roles
    WHERE rolname = 'data_analyst';""")
cu.fetchall()

[(False, True, False, False, '********')]

In [25]:
cu.execute("""SELECT rolsuper, rolcanlogin, rolcreaterole, rolcreatedb, rolpassword
    FROM pg_roles
    WHERE rolname = 'data_scientist';""")
cu.fetchall()

[(False, True, False, False, '********')]

In [26]:
cu.execute("""SELECT rolsuper, rolcanlogin, rolcreaterole, rolcreatedb, rolpassword
    FROM pg_roles
    WHERE rolname = 'dq';""")
cu.fetchall()

[(True, True, True, True, '********')]

After cheking pg_roles and information_schema.table_privileges tables, it appears that:

- Table privileges are correctly granted for each group.
- Database privileges have been correctly limited for each group.

The connection can now be closed

In [27]:
conn.close()